In [ ]:
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
import keras as k
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_excel('/kaggle/input/COVID-19_bd-2 (2) (2).xlsx')

In [ ]:
df

In [ ]:
df.drop(['Unnamed: 11'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
#df['traveled country'].value_counts()

In [ ]:
df['heart disease'] = df['heart disease'].map({'yes': 1, 'no': 0})
df['lung disease'] = df['lung disease'].map({'yes': 1, 'no': 0})
df['diabetes'] = df['diabetes'].map({'yes': 1, 'no': 0})
df['fever'] = df['fever'].map({'yes': 1, 'no': 0})
df['cough'] = df['cough'].map({'yes': 1, 'no': 0})
df['shortness of breathe'] = df['shortness of breathe'].map({'yes': 1, 'no': 0})
df['pressure in the chest'] = df['pressure in the chest'].map({'yes': 1, 'no': 0})
df['close contact with corona patient'] = df['close contact with corona patient'].map({'yes': 1, 'no': 0})
#df['corona patient'] = df['corona patient'].map({'corona': 1, 'flu': 2, 'cold':3})
df['sex'] = df['sex'].map({'male': 1, 'female': 0})
#df['traveled country'] = df['traveled country'].map({'china': 1, 'singapore': 2,'usa': 3, 'south korea': 4,'italy': 5, 'germany': 6,
                                                     #'switzerland': 7, 'spain': 8,'thailand': 9,'egypt':10,'belgium':11,'lebanon':12,
                                                     #'iraq':13,'iran':14,'afganistan':15,'kuwait':16,'algeria':17,'austria':18,'nepal':19,
                                                     #'malaysia':20,'sri lanka':21,'india':22,'sweden':23,'canada':24,'netherlands':25,
                                                     #'brazil':26,'greece':27,'israel':28,'russia':29,'chile':30,'mexico':31,'belarus':32,
                                                     #'cyprus':33,'turkey':34,'honduras':35,'kenya':36,'uk':37,'costa rica':38,'armenia':39,
                                                    #'ecuador':40,'bosnia':41,'azerbaijan':42,'peru':43,'panama':44,'bulgeria':45,'portugal':46})

df['traveled country'] = df['traveled country'].map({'china': 2, 'singapore': 8,'usa': 3, 'south korea': 5,'italy': 1, 'germany': 9,
                                                     'switzerland': 7, 'spain': 4,'thailand': 6,'egypt':10,'belgium':11,'lebanon':12,
                                                     'iraq':11,'iran':11,'afganistan':12,'kuwait':12,'algeria':12,'austria':12,'nepal':12,
                                                     'malaysia':10,'sri lanka':12,'india':12,'sweden':10,'canada':10,'netherlands':12,
                                                     'brazil':12,'greece':12,'israel':12,'russia':12,'chile':12,'mexico':12,'belarus':12,
                                                     'cyprus':12,'turkey':12,'honduras':12,'kenya':12,'uk':10,'costa rica':12,'armenia':12,
                                                    'ecuador':12,'bosnia':12,'azerbaijan':12,'peru':12,'panama':12,'bulgeria':12,'portugal':11})
                       

In [ ]:
df

In [ ]:
df.info()

In [ ]:
#Split the data
X = df.drop(["disease",'flu','cold','corona'], axis=1)
#y_corona = df["corona"]
#y_flu = df['flu']
#y_cold = df['cold']
y = df[['corona','flu','cold']]

In [ ]:
X

In [ ]:
y

In [ ]:
#Feature Scaling
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names] = x_scaler.transform(X)

In [ ]:
X

In [ ]:
#Split the data into 80% training and 20% testing 
X_train,  X_test, y_train, y_test = train_test_split(
        X, y, test_size= 0.2, shuffle=True)

In [ ]:
#Build The model
model = Sequential()
model.add(Dense(512, input_dim=len(X.columns),kernel_initializer=k.initializers.random_normal(seed=13), activation="relu"))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation="softmax"))

In [ ]:
#Compile the model
model.compile(loss='binary_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])

In [ ]:
#Train the model
history = model.fit(X_train, y_train, 
                    epochs=1000, 
                    batch_size=X_train.shape[0]) 

In [ ]:
#Save the model
model.save("covid-19.model")

In [ ]:
#Visualize the models accuracy and loss
plt.plot(history.history["accuracy"])
plt.plot(history.history["loss"])
plt.title("model accuracy & loss")
plt.ylabel("accuracy and loss")
plt.xlabel("epoch")
plt.legend(['acc', 'loss'], loc='lower right')
plt.show()

In [ ]:
model

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_test

In [ ]:
y_test.reset_index(drop=True,inplace=True)

In [ ]:
prediction = model.predict_classes(X_test)

prediction = pd.DataFrame(prediction)

prediction.columns = ['prediction']

In [ ]:
map_dict = {0:'corona',1:'flu',2:'cold'} 
#y_test["test"] = prediction.loc[:,0].map(map_dict)
y_test['predicted'] = prediction


y_test

In [ ]:
map_dict = {0:'corona',1:'flu',2:'cold'} 
y_test["predicted"] = y_test["predicted"].map(map_dict)
y_test

In [ ]:
y_test.to_excel("output.xlsx")